# TeddyNote Parser API 클라이언트 테스트

이 노트북은 `parser_client.py`에 구현된 `TeddyNoteParserClient` 클래스를 사용하여 TeddyNote Parser API와 상호작용하는 과정을 보여줍니다.

주요 단계:
1. 클라이언트 초기화 및 API 연결 확인
2. PDF 파일 업로드 및 파싱 작업 요청
3. 작업 상태 확인
4. 완료된 파싱 결과 다운로드

In [3]:
%%capture
!pip install -U teddynote-parser-api pandas langchain

In [1]:
import time

import pandas as pd
from IPython.display import display

## 환경 설정

In [2]:
import os
from pathlib import Path

# API 서버 설정
API_URL = "http://localhost:9990"  # API 서버 주소

# PDF 파일 경로
PDF_FILE_PATH = "data/2210.03629v3.pdf"  # PDF 파일 경로

# 결과 저장 디렉토리
OUTPUT_DIR = "client_test_results"
RESULTS_DIR = Path(OUTPUT_DIR)
RESULTS_DIR.mkdir(exist_ok=True)  # 결과 저장 디렉토리 생성

# API 키 설정 (환경 변수에서 로드하거나 직접 지정)
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY", "your_upstage_api_key")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "your_openai_api_key")

## 1. 클라이언트 초기화 및 설정

In [3]:
import logging
from teddynote_parser_client.client import TeddyNoteParserClient

# 로깅 설정
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger("teddynote_parser_client")

# TeddyNoteParserClient 인스턴스 생성
client = TeddyNoteParserClient(
    api_url=API_URL,  # API 서버 URL
    upstage_api_key=UPSTAGE_API_KEY,  # UPSTAGE LLM API 키
    openai_api_key=OPENAI_API_KEY,  # OpenAI API 키 (대체 LLM으로 사용)
    batch_size=50,  # 한 번에 처리할 PDF 페이지 수
    test_page=None,  # 테스트용 페이지 제한 (None: 전체 페이지 처리)
    language="Korean",  # 문서 언어 설정
    include_image=True,  # 결과에 이미지 포함 여부
    logger=logger,  # 로깅에 사용할 로거
)

서버에 정상 동작중인지 확인합니다.

In [4]:
# API 서버 상태 확인
try:
    health_status = client.health_check()
    print(f"✅ API 서버가 정상적으로 실행 중입니다. 상태: {health_status}")
    api_available = True
except Exception as e:
    print(f"❌ API 서버에 접속할 수 없습니다: {e}")
    print("API 서버가 실행 중인지 확인해주세요.")
    api_available = False

2025-03-10 03:53:49,860 - teddynote_parser_client - INFO - API 서버가 정상적으로 응답했습니다.


✅ API 서버가 정상적으로 실행 중입니다. 상태: {'status': 'ok', 'timestamp': '2025-03-10T03:53:49.858499'}


## 2. PDF 파일 업로드 및 파싱 작업 요청

In [5]:
# PDF 파일 파싱 요청
if api_available:
    try:
        print(f"📄 파일 '{PDF_FILE_PATH}'에 대한 파싱 작업을 요청합니다...")
        # pdf_path: 파싱할 PDF 파일 경로
        # batch_size: 한 번에 처리할 페이지 수(기본값: 30, 최대값: 100)
        # language: Entity 파싱 언어
        # test_page: 테스트용으로 처음 ~ 지정한 페이지까지만 처리(기본값: None - 모든 페이지 처리)
        # include_image: 결과에 이미지 포함 여부(기본값: True)
        parse_result = client.parse_pdf(
            pdf_path=PDF_FILE_PATH,
            batch_size=50,
            language="English",
            test_page=5,
            include_image=True,
        )

        job_id = parse_result["job_id"]
        print(f"✅ 파싱 작업이 시작되었습니다!")
        print(f"📝 작업 ID: {job_id}")
        print(f"📝 상태: {parse_result['status']}")
        print(f"📝 메시지: {parse_result['message']}")
    except Exception as e:
        print(f"❌ 파싱 작업 요청 실패: {e}")
        job_id = None

2025-03-10 03:53:51,338 - teddynote_parser_client - INFO - 파일 '2210.03629v3.pdf'에 대한 파싱 작업 요청 중...
2025-03-10 03:53:51,339 - teddynote_parser_client - INFO - 파싱 옵션: 언어=English, 이미지 포함=True, 배치 크기=50, 처리 페이지 수=5
2025-03-10 03:53:51,359 - teddynote_parser_client - INFO - 파싱 작업이 시작되었습니다. 작업 ID: 3d541e4d-21ba-4b1c-9c69-d4c82d346422


📄 파일 'data/2210.03629v3.pdf'에 대한 파싱 작업을 요청합니다...
✅ 파싱 작업이 시작되었습니다!
📝 작업 ID: 3d541e4d-21ba-4b1c-9c69-d4c82d346422
📝 상태: pending
📝 메시지: PDF 파싱 작업이 시작되었습니다.


## 3. 작업 상태 확인

In [6]:
# 작업 상태 확인 및 완료 대기
if "job_id" in locals() and job_id:
    print(f"작업 ID: {job_id}에 대한 상태를 확인하고 완료될 때까지 대기합니다...")

    try:
        # 비동기 작업 완료 대기 (최대 60초, 2초 간격으로 확인)
        final_status = client.wait_for_job_completion(
            job_id, check_interval=2, max_attempts=30
        )

        if final_status["status"] == "completed":
            print("\n작업 최종 정보:")
            print(f"📝 파일명: {final_status['filename']}")
            print(
                f"📝 생성 시간: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(final_status['created_at']))}"
            )
            print(
                f"📝 완료 시간: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(final_status['completed_at']))}"
            )
            print(
                f"📝 처리 시간: {final_status['completed_at'] - final_status['created_at']:.2f}초"
            )
            print(f"📝 ZIP 파일: {final_status['zip_filename']}")
        elif final_status["status"] == "failed":
            print(f"❌ 작업 실패: {final_status.get('error', '알 수 없는 오류')}")
    except TimeoutError as e:
        print(f"⚠️ 작업 대기 시간 초과: {e}")
    except Exception as e:
        print(f"❌ 작업 상태 확인 중 오류 발생: {e}")

2025-03-10 03:53:53,731 - teddynote_parser_client - INFO - 작업 ID '3d541e4d-21ba-4b1c-9c69-d4c82d346422'의 완료 대기 중...
2025-03-10 03:53:53,731 - teddynote_parser_client - INFO - 상태 확인 간격: 2초, 최대 시도 횟수: 30회
2025-03-10 03:53:53,736 - teddynote_parser_client - INFO - 작업 ID '3d541e4d-21ba-4b1c-9c69-d4c82d346422'의 현재 상태: processing


작업 ID: 3d541e4d-21ba-4b1c-9c69-d4c82d346422에 대한 상태를 확인하고 완료될 때까지 대기합니다...


2025-03-10 03:53:55,748 - teddynote_parser_client - INFO - 작업 ID '3d541e4d-21ba-4b1c-9c69-d4c82d346422'의 현재 상태: processing
2025-03-10 03:53:57,768 - teddynote_parser_client - INFO - 작업 ID '3d541e4d-21ba-4b1c-9c69-d4c82d346422'의 현재 상태: processing
2025-03-10 03:53:59,776 - teddynote_parser_client - INFO - 작업 ID '3d541e4d-21ba-4b1c-9c69-d4c82d346422'의 현재 상태: processing
2025-03-10 03:54:01,787 - teddynote_parser_client - INFO - 작업 ID '3d541e4d-21ba-4b1c-9c69-d4c82d346422'의 현재 상태: completed
2025-03-10 03:54:01,787 - teddynote_parser_client - INFO - 작업 ID '3d541e4d-21ba-4b1c-9c69-d4c82d346422'가 completed 상태로 완료되었습니다.



작업 최종 정보:
📝 파일명: 2210.03629v3.pdf
📝 생성 시간: 2025-03-10 03:53:51
📝 완료 시간: 2025-03-10 03:53:59
📝 처리 시간: 8.51초
📝 ZIP 파일: result/3d541e4d-21ba-4b1c-9c69-d4c82d346422_20250310_035359.zip


## 4. 파싱 결과 다운로드

In [7]:
# 파싱 결과 다운로드 및 압축 해제
if (
    "job_id" in locals()
    and job_id
    and "final_status" in locals()
    and final_status["status"] == "completed"
):
    try:
        # 결과 다운로드 및 압축 해제
        zip_path, extract_path = client.download_result(
            job_id=job_id, save_dir=RESULTS_DIR, extract=True, overwrite=True
        )

        print(f"✅ 파싱 결과가 성공적으로 다운로드되었습니다: {zip_path}")
        print(f"✅ ZIP 파일이 성공적으로 압축 해제되었습니다: {extract_path}")
        print(f"\n파싱 결과 디렉토리: {extract_path}")
    except Exception as e:
        print(f"❌ 결과 다운로드 중 오류 발생: {e}")
        extract_path = None

2025-03-10 03:54:05,517 - teddynote_parser_client - INFO - 작업 ID '3d541e4d-21ba-4b1c-9c69-d4c82d346422'의 현재 상태: completed
2025-03-10 03:54:05,518 - teddynote_parser_client - INFO - 작업 ID '3d541e4d-21ba-4b1c-9c69-d4c82d346422'의 결과 다운로드 중...
2025-03-10 03:54:05,529 - teddynote_parser_client - INFO - 결과가 성공적으로 다운로드되었습니다: client_test_results/3d541e4d-21ba-4b1c-9c69-d4c82d346422_20250310035405.zip
2025-03-10 03:54:05,533 - teddynote_parser_client - INFO - ZIP 파일 'client_test_results/3d541e4d-21ba-4b1c-9c69-d4c82d346422_20250310035405.zip'의 압축을 'client_test_results/3d541e4d-21ba-4b1c-9c69-d4c82d346422'에 해제했습니다.
2025-03-10 03:54:05,534 - teddynote_parser_client - INFO - ZIP 파일의 압축이 성공적으로 해제되었습니다: client_test_results/3d541e4d-21ba-4b1c-9c69-d4c82d346422


✅ 파싱 결과가 성공적으로 다운로드되었습니다: client_test_results/3d541e4d-21ba-4b1c-9c69-d4c82d346422_20250310035405.zip
✅ ZIP 파일이 성공적으로 압축 해제되었습니다: client_test_results/3d541e4d-21ba-4b1c-9c69-d4c82d346422

파싱 결과 디렉토리: client_test_results/3d541e4d-21ba-4b1c-9c69-d4c82d346422


## 5. 작업 목록 조회

In [8]:
# 작업 목록 조회
if api_available:
    try:
        jobs = client.list_all_jobs()["jobs"]

        print(f"📋 총 {len(jobs)}개의 작업이 있습니다.\n")

        # 작업 정보를 DataFrame으로 변환
        if jobs:
            job_data = []
            for job in jobs:
                created_at = time.strftime(
                    "%Y-%m-%d %H:%M:%S", time.localtime(job["created_at"])
                )
                completed_at = (
                    time.strftime(
                        "%Y-%m-%d %H:%M:%S", time.localtime(job["completed_at"])
                    )
                    if job["completed_at"]
                    else "N/A"
                )

                job_data.append(
                    {
                        "Job ID": job["job_id"],
                        "Status": job["status"],
                        "Filename": job["filename"],
                        "Created At": created_at,
                        "Completed At": completed_at,
                    }
                )

            df = pd.DataFrame(job_data)
            display(df)
    except Exception as e:
        print(f"❌ 작업 목록 조회 중 오류 발생: {e}")

2025-03-10 03:54:17,673 - teddynote_parser_client - INFO - 모든 작업 목록 조회 중...
2025-03-10 03:54:17,678 - teddynote_parser_client - INFO - 총 1 개의 작업이 조회되었습니다.


📋 총 1개의 작업이 있습니다.



,Job ID,Status,Filename,Created At,Completed At
0,3d541e4d-21ba-4b1c-9c69-d4c82d346422,completed,2210.03629v3.pdf,2025-03-10 03:53:51,2025-03-10 03:53:59


In [12]:
import pickle


def load_documents_from_pkl(filepath):
    """
    Pickle 파일에서 Langchain Document 리스트를 불러오는 함수

    Args:
        filepath: 원본 파일 경로 (예: path/to/filename.pdf)
    Returns:
        Langchain Document 객체 리스트
    """
    # 확장자 제거하고 절대 경로로 변환
    abs_path = os.path.abspath(filepath)
    base_path = os.path.splitext(abs_path)[0]
    pkl_path = f"{base_path}.pkl"

    with open(pkl_path, "rb") as f:
        documents = pickle.load(f)
    return documents

In [10]:
import glob
from pathlib import Path

# extract_path 디렉토리에서 모든 .pkl 파일 찾기
pkl_files = glob.glob(str(Path(extract_path) / "*" / "*.pkl"))

if not pkl_files:
    print("❌ extract_path에서 .pkl 파일을 찾을 수 없습니다.")
else:
    # 모든 .pkl 파일에서 문서 로드
    all_documents = []
    for pkl_file in pkl_files:
        print(f"📄 {pkl_file} 파일 로드 중...")  # 한국어 코멘트
        documents = load_documents_from_pkl(pkl_file)
        all_documents.extend(documents)
    
    print(f"✅ 총 {len(all_documents)}개의 문서가 로드되었습니다.")

📄 client_test_results/3d541e4d-21ba-4b1c-9c69-d4c82d346422/3d541e4d-21ba-4b1c-9c69-d4c82d346422/3d541e4d-21ba-4b1c-9c69-d4c82d346422_2210.03629v3.pkl 파일 로드 중...
✅ 총 25개의 문서가 로드되었습니다.


In [11]:
all_documents

[Document(metadata={'page': 0, 'source': 'uploads/3d541e4d-21ba-4b1c-9c69-d4c82d346422_2210.03629v3.pdf'}, page_content='# REAC T: SYNERGIZING REASONING AND ACTING IN\nLANGUAGE MODELS\nShunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n1Department of Computer Science, Princeton University\n2Google Research, Brain team\n1{shunyuy,karthikn}@princeton.edu\n2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\nABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action\nplan generation) have primarily been studied as separate topics. In this paper, we\nexplore the use of LLMs to generate both reasoning traces and task-speciﬁc actions\nin an interleaved manner, allowing for greater synergy between the two: reasoning\ntraces help the model induce, track, and up